In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "ProgressMeter",
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
long_reads = sort(filter(x -> occursin(r".badread.nanopore2023.", x) && !occursin("filtlong", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
fasta_files = sort(filter(x -> occursin(r"\.fna$", x) && !occursin(".vcf", x) && !occursin("normalized", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
ProgressMeter.@showprogress for in_fastq in long_reads
    out_fastq = replace(in_fastq, ".fq.gz" => ".filtlong.fq.gz")
    if !isfile(out_fastq)
        @time run(Mycelia.filter_long_reads(in_fastq = in_fastq))
        @assert isfile(out_fastq)
    else
        @info "$(out_fastq) already present"
    end
end